In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import mean_squared_error

In [2]:
jan_data = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
feb_data = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

print(f"Q1. Columns in january: {jan_data.shape[1]}")

Q1. Columns in january: 19


In [ ]:
jan_data['duration'] = jan_data['tpep_dropoff_datetime'] - jan_data['tpep_pickup_datetime']
feb_data['duration'] = feb_data['tpep_dropoff_datetime'] - feb_data['tpep_pickup_datetime']

In [ ]:
jan_data['duration'] = jan_data['duration'].dt.total_seconds() / 60
feb_data['duration'] = feb_data['duration'].dt.total_seconds() / 60

In [3]:
print(f"Q2. Standard Deviation of trips duration in January: {jan_data.duration.std()}")

Q2. Standard Deviation of trips duration in January: 42.594351241920904


In [ ]:
# Outliers

In [ ]:
jan_data.duration.describe()

In [ ]:
jan_data.duration.mean()

In [ ]:
jan_data_cleaned = jan_data[(jan_data.duration >= 1) & (jan_data.duration <= 60)]
feb_data_cleaned = feb_data[(feb_data.duration >= 1) & (feb_data.duration <= 60)]

In [4]:
print(f"Q3. Fraction of the records left after dropping outliers: {(jan_data_cleaned.shape[0]/jan_data.shape[0]) * 100}")

Q3. Fraction of the records left after dropping outliers: 98.1220282212598


In [ ]:
# One Hot Encoding
categorical = ['PULocationID','DOLocationID']
jan_data_cleaned[categorical] = jan_data_cleaned[categorical].astype(str)
feb_data_cleaned[categorical] = feb_data_cleaned[categorical].astype(str)

In [ ]:
train_dicts = jan_data_cleaned[categorical].to_dict(orient='records')
val_dicts = feb_data_cleaned[categorical].to_dict(orient='records')

In [ ]:
train_dicts

In [ ]:
val_dicts

In [ ]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

X_val = dv.transform(val_dicts)

In [5]:
X_train.shape

<ipython-input-5-4a7c3cbce878>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_data_cleaned[categorical] = jan_data_cleaned[categorical].astype(str)
<ipython-input-5-4a7c3cbce878>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feb_data_cleaned[categorical] = feb_data_cleaned[categorical].astype(str)


(3009173, 515)

In [6]:
X_val.shape

(2855951, 515)

In [7]:
print(f"Q4. What's the dimensionality of this matrix (number of columns)? : {X_train.shape[1]}")

Q4. What's the dimensionality of this matrix (number of columns)? : 515


In [8]:
# Training model

In [9]:
target = 'duration'
y_train = jan_data_cleaned[target].values
y_val = feb_data_cleaned[target].values

In [10]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [11]:
y_pred = lr.predict(X_train)
mse = mean_squared_error(y_train,y_pred)
rmse = np.sqrt(mse)

In [12]:
print(f"Q5. RMSE on train: {rmse}")

Q5. RMSE on train: 7.649261932106969


In [14]:
#Rmse on train
y_pred = lr.predict(X_val)
mse = mean_squared_error(y_val,y_pred)
rmse = np.sqrt(mse)

In [15]:
print(f"Q6. RMSE on validation: {rmse}")

Q5. RMSE on train: 7.811818743246608
